In [8]:
%load_ext autoreload
%autoreload 2
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'ERROR'
os.environ['FILE_LOG_LEVEL'] = 'DEBUG'
os.environ['PROPAGATE_TO_ROOT_LOGGER'] = 'False'
os.environ['PROPAGATE_TO_ROOT_LOGGER'], os.environ['STREAM_LOG_LEVEL']
from trade.assets.Stock import Stock
from trade.assets.Option import Option
from trade.helpers.Context import Context, clear_context
from trade.helpers.helper import change_to_last_busday, is_USholiday, is_busday, setup_logger
from scipy.stats import percentileofscore
from dbase.DataAPI.ThetaData import list_contracts
import pandas as pd
import numpy as np
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sys.path
from dbase.DataAPI.ThetaData import list_contracts
list_contracts('AAPL', '2025-01-13')

,root,expiration,strike,right
0,AAPL,20250919,215.0,P
1,AAPL,20250919,215.0,C
2,AAPL,20251017,215.0,P
3,AAPL,20250321,350.0,C
4,AAPL,20250124,220.0,P
...,...,...,...,...
1278,AAPL,20250131,217.5,C
1279,AAPL,20250718,215.0,C
1280,AAPL,20250718,215.0,P
1281,AAPL,20250815,215.0,P


In [3]:
class RiskManager:
    def __init__(self,
                 bars,
                 events,
                 initial_capital,
                 ):
        self.bars = bars
        self.events = events
        self.initial_capital = initial_capital
        self.symbol_list = self.bars.symbol_list


In [4]:
def generate_option_order(
        direction,
        dte,
        relative_strike,
):
    pass



In [5]:
## Why is AMZN during cheap vol returning two?
def chain_details(date, ticker, tgt_dte, tgt_moneyness, right = 'P', moneyness_width = 0.15):
    return_dataframe = pd.DataFrame()
    errors = {}
    if not (is_USholiday(date) and not is_busday(date)):
        try:
            print(date, ticker)
            ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
            ## We will need to get two rows. 
            with Context(end_date = date):
                start_time = time.time()
                Stock_obj = Stock(ticker)
                end_time = time.time()
                print(f"Time taken to get stock object: {end_time-start_time}")
                Option_Chain = Stock_obj.option_chain()
                Option_Chain_Filtered = Option_Chain[Option_Chain[right.upper()] == True]
                Spot = Stock_obj.spot(ts = False)
                Spot = list(Spot.values())[0]
                if right == 'P':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.index.get_level_values('strike')/Spot
                elif right == 'C':
                    Option_Chain_Filtered['relative_moneyness']  = Spot/Option_Chain_Filtered.index.get_level_values('strike')
                else:
                    raise ValueError(f'Right dne. recieved {right}')
                Option_Chain_Filtered['moneyness_spread'] = (tgt_moneyness-Option_Chain_Filtered['relative_moneyness'])**2
                Option_Chain_Filtered['dte_spread'] = (Option_Chain_Filtered.index.get_level_values('DTE')-tgt_dte)**2
                Option_Chain_Filtered.sort_values(by=['dte_spread','moneyness_spread'], inplace = True)
                Option_Chain_Filtered = Option_Chain_Filtered.loc[Option_Chain_Filtered['dte_spread'] == Option_Chain_Filtered['dte_spread'].min()]
                option_details = Option_Chain_Filtered[(Option_Chain_Filtered['relative_moneyness'] >= tgt_moneyness-moneyness_width) & (Option_Chain_Filtered['relative_moneyness'] <= tgt_moneyness+moneyness_width)]
                option_details['build_date'] = date
                option_details['ticker'] = ticker
                option_details['Spot'] = Spot
                option_details['moneyness'] = tgt_moneyness
                option_details['TGT_DTE'] = tgt_dte
                option_details['q'] = Stock_obj.div_yield()
                option_details['r'] = Stock_obj.rf_rate
                option_details.reset_index(inplace = True)
                option_details.set_index('build_date', inplace = True)
                return_dataframe = pd.concat([return_dataframe, option_details])
            clear_context()
            return_dataframe.drop_duplicates(inplace = True)

        except Exception as e:
            raise
            errors[date] = e
            return errors
        return return_dataframe.sort_values('relative_moneyness', ascending=False),Stock_obj
    else:
        return None, errors
    

details, obj = chain_details('2024-03-12', 'AAPL', 365, 1, moneyness_width = 0.20)

2024-03-12 AAPL
Time taken to get stock object: 4.578587770462036


In [24]:
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.1},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.1}
            ],
            'name': 'vertical_spread'
        }

Steps to producing an order:

S1: RM recieves order settings from PM
S2: RM produces a dataframe of potential options based on settings (if two legs produce two dataframes)
S3: RM assesses if option passes all checks
    C1: Minimum Available close
    C2: Liquidity (Open Interest)
    C2.5: (for Spreads only) Ensure both legs are not the same
    Optional, to extend:
    C3: Bid-Ask Spread
    
S4: Return picked order to portfolio manager, which places the order. 
Example:
    {'long': [optionid or {'strike', 'exp'}], 'short' : []}

In [7]:
class OptionSettingsDict(dict):
    


SyntaxError: invalid syntax (3277259435.py, line 1)

In [40]:
from trade.helpers.helper import generate_option_tick
option_keys = {}
def import_option_keys():
    global option_keys
    import json
    with open('/Users/chiemelienwanisobi/cloned_repos/QuantTools/trade/assets/option_key.json', 'r') as f:
        option_keys = json.load(f)

def save_option_keys(key, info):
    import json
    global option_keys
    import_option_keys()
    if key not in option_keys.keys():
        option_keys[key] = info    
        with open('/Users/chiemelienwanisobi/cloned_repos/QuantTools/trade/assets/option_key.json', 'w') as f:
            json.dump(option_keys, f)
    else:
        print(f"{key} already exists in option_keys")

In [65]:
generate_option_tick('AAPL','P' ,'2025-03-21', 150.0)
# save_option_keys(generate_option_tick('AAPL','P' ,'2025-03-21', 150.0), {'ticker':'AAPL', 'put_call':'P', 'exp_date':'2025-03-21', 'strike':150.0})

'AAPL20250321000150P'

In [57]:
opt = Option(**option_keys['AAPL20250321000150P'])

Updating db rates data


In [39]:
import_option_keys()
option_keys

{}